I recently change the domain for my Microblog from `myles.life` to `myles.social` and need to redirect the older URLs using Vercel.

In [ ]:
import datetime
from pathlib import Path
import json
from urllib.parse import urlparse
from lazymyles.download_file import download_file

In [2]:
ROOT_PATH = Path.cwd()
DATA_PATH = ROOT_PATH / "data"
RAW_DATA_PATH = DATA_PATH / "raw"
PROCESSED_DATA_PATH = DATA_PATH / "processed"

In [3]:
FEED_JSON_PATH = download_file(
    "https://raw.githubusercontent.com/myles/myles.social/master/feed.json",
    output_path=RAW_DATA_PATH,
    file_name=f"feed-{datetime.date.today()}.json",
)

In [4]:
with FEED_JSON_PATH.open() as file_obj:
    raw_feed_json = file_obj.read()

feed_json = json.loads(raw_feed_json)
feed_json["title"]

"Myles' Life"

In [5]:
vercel_config = {}

In [6]:
def feed_json_item_to_vercel_redirect(item: dict) -> dict:
    url = urlparse(item["url"])
    return {
        "source": url.path,
        "destination": f"https://myles.social{url.path}",
        "permanent": True,
    }


feed_json_item_to_vercel_redirect({"url": "https://myles.life/2022/11/01/204923.html"})

{'source': '/2022/11/01/204923.html',
 'destination': 'https://myles.social/2022/11/01/204923.html',
 'permanent': True}

In [7]:
vercel_config["redirects"] = [
    feed_json_item_to_vercel_redirect(item) for item in feed_json["items"]
]

In [10]:
raw_vercel_config = json.dumps(vercel_config, sort_keys=True, indent=4)

VERCEL_CONFIG_PATH = PROCESSED_DATA_PATH / "vercel.json"

with VERCEL_CONFIG_PATH.open("w+") as file_obj:
    file_obj.write(raw_vercel_config)